In [200]:
import QUANTAXIS as QA
import pandas as pd
from datetime import datetime


CODES = [
    '601816', # 美凯龙
    '601816', # 中国高铁
    '601318', # 中国平安
    '600864', # 哈投股份
    '300708', # 聚灿光电
    '300347', # 泰格医药
    '300152', # 科融环境
    '002713', # 东易日盛
    '002466', # 天齐锂业
    '002351', # 漫步者
    '002049', # 紫光国微
    '000400', # 许继电气
    '000723', # 美锦能源
    '159977', # 创业板TH
    '159995', # 芯片ETF
]
CODES = ['000723', '002466', '601318']
now = datetime.now().strftime('%Y-%m-%d')

In [201]:
def ene(data):
    M1 = 11
    M2 = 9
    N = 10

    data['MA%s' % N] = round(data['close'].rolling(N).mean(), 3)
    data['ENE_UP'] = (1 + M1 / 100.0) * data['MA%s' % N]
    data['ENE_LOW'] = (1 - M2 / 100.0) * data['MA%s' % N]
    data['ENE'] = round((data['ENE_UP'] + data['ENE_LOW']) / 2, 3)
    data['ENE_UP'] = round(data['ENE_UP'], 3)
    data['ENE_LOW'] = round(data['ENE_LOW'], 3)

In [282]:
def ene_buy_sell(data):
    buy_position = data['ENE_LOW'] > data['low']
    data.loc[buy_position[buy_position == True].index, 'buy_sell'] = 'buy'
    sell_position = data['ENE_UP'] < data['high']
    data.loc[sell_position[(sell_position == True) & (sell_position.shift() == False)].index, 'buy_sell'] = 'sell'

In [284]:
data = QA.QA_fetch_stock_day_adv(CODES, '2000-01-01', now)

In [285]:
qfq_data = data.to_qfq()
# qfq_data.data.loc[(slice(None), '300027'),:]
# qfq_data.data.loc[(slice(None), CODES),:]

In [286]:
code = '000723'
data = qfq_data.data.loc[(slice(None), code), :].copy()
data.reset_index(inplace=True)
# final_data = data[['date', 'open', 'high', 'low', 'close', 'volume', 'amount']].copy()
final_data = data[['date', 'high', 'low', 'close']].copy()

In [287]:
ene(final_data)
ene_buy_sell(final_data)
final_data = final_data[['date', 'high', 'close', 'buy_sell']].copy()

In [288]:
# N天内最高价、最高收盘价
days = [5, 13, 21, 42, 60]
for day in days:
    # final_data['HH%s' % day] = final_data['high'].rolling(day).max().shift(-day)
    # final_data['HHR%s' % day] = (final_data['HH%s' % day] / final_data['close'] - 1)
    final_data['HC%s' % day] = final_data['close'].rolling(day).max().shift(-day)
    final_data['HCR%s' % day] = (final_data['HC%s' % day] / final_data['close'] - 1)

In [290]:
final_data.to_csv('ene_%s.csv' % code, sep='\t', index=False)

In [303]:
pd.set_option('expand_frame_repr', False)
for t, group in final_data.groupby('buy_sell'):
    if t != 'buy':
        continue
    print(group[['HCR%s' % day for day in days]].describe())
    print('N天内上涨概率\t买入总次数\t买入后上涨次数\t成功率')
    for day in days:
        label = 'HCR%s' % day
        total = group.shape[0]
        succ = group[group[label] > 0].shape[0]
        ratio = float(succ) / total
        print(f'{day}\t{total}\t{succ}\t{ratio}')

             HCR5       HCR13       HCR21       HCR42       HCR60
count  376.000000  376.000000  376.000000  370.000000  368.000000
mean     0.043342    0.102590    0.139875    0.223226    0.281685
std      0.073090    0.152613    0.187045    0.277424    0.399947
min     -0.100204   -0.100204   -0.100204   -0.100204   -0.100204
25%     -0.001052    0.017739    0.029231    0.059262    0.065733
50%      0.031579    0.065435    0.088948    0.140449    0.171470
75%      0.079583    0.133490    0.187557    0.328460    0.376823
max      0.466790    1.034156    1.162571    1.745387    4.786819
N天内上涨概率	买入总次数	买入后上涨次数	成功率
5	377	278	0.7374005305039788
13	377	309	0.8196286472148541
21	377	321	0.8514588859416445
42	377	335	0.8885941644562334
60	377	336	0.8912466843501327
